In [ ]:
sed 's/,/   /g' Pool10_Barcodes.csv > Pool10_Barcodes.txt

In [ ]:
scp Pool10_Barcodes.txt sarahgw@amphiprion.deenr.rutgers.edu:/local/home/sarahgw/WGS/Sequences/Pool10/

In [ ]:
mkdir $(awk '{print $1}' /local/home/sarahgw/WGS/Sequences/Pool10/Pool10_Barcodes.txt)

In [ ]:
for dir in $(ls); do cd $dir; cp /local/shared/pinsky_lab/sequencing/hiseq_2018_03_06_SEQ27/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-10-from-flowcell-HW5LWBCXY-on-2018-03-06_Read_4_passed_filter.fastq.gz /local/shared/pinsky_lab/sequencing/hiseq_2018_03_06_SEQ27/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-10-from-flowcell-HW5LWBCXY-on-2018-03-06_Read_1_passed_filter.fastq.gz  . ; cd ..; done

In [ ]:
gunzip */*.fastq.gz

In [ ]:
~/Notebooks/WNS_WGS/Bioinfo_pipeline.sh 